In [1]:
import numpy as np
from transformers import pipeline

/Users/izelteker/Documents/UB Data Science/DL/HuggingFace/.env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/izelteker/Documents/UB Data Science/DL/HuggingFace/.env/lib/python3.13/site-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [2]:
from datasets import load_dataset
import os

os.makedirs("data", exist_ok=True)
ds = load_dataset("deadprogram/clothes-with-class")

/Users/izelteker/Documents/UB Data Science/DL/HuggingFace/.env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
img1 = ds["train"][2]["image"]
text1 = ds["train"][2]["description"]

NameError: name 'ds' is not defined

In [4]:
ds_blip = (
    ds
    .rename_column("description", "text")
    .select_columns(["image", "text"])
)


In [6]:
cloth_sample = ds_blip.shuffle(seed=42).select(range(1000))
# Peek at the first few examples
cloth_sample[:3]

{'image': [<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1166x1750>,
  <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1166x1750>,
  <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1166x1750>],
 'text': ['solid black off-the-shoulder blouse in a crinkled weave with wide dolman sleeves and visible seams front and back elastication around the top sleeves and hem unlined',
  'all over pattern black short-sleeved shirt in a patterned viscose weave with a resort collar french front and straight-cut hem with short slits in the sides',
  'solid white sleeveless blouse in woven fabric with a v-neck concealed button placket and rounded hem']}

In [7]:
cloth_sample["image"][1].size

(1166, 1750)

In [8]:
# original dataset
dataset = ds_blip  # 88400 samples

# first, split off test set (e.g., 10%)
split1 = dataset.train_test_split(test_size=0.1, seed=42)
train_val_dataset = split1['train']  # 90%
test_dataset = split1['test']        # 10%

# then, split train_val into train & validation (e.g., 80/20 of 90%)
split2 = train_val_dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = split2['train']       # 72%
val_dataset = split2['test']          # 18%


In [9]:
from datasets import DatasetDict
final_dataset = DatasetDict({
    "train": train_dataset,
    "validation": val_dataset,
    "test": test_dataset
})


In [10]:
print(final_dataset)


DatasetDict({
    train: Dataset({
        features: ['image', 'text'],
        num_rows: 62928
    })
    validation: Dataset({
        features: ['image', 'text'],
        num_rows: 15732
    })
    test: Dataset({
        features: ['image', 'text'],
        num_rows: 8740
    })
})


In [11]:
#final_dataset.save_to_disk("cloth_ds")

Saving the dataset (4/4 shards): 100%|██████████| 8740/8740 [00:24<00:00, 350.75 examples/s] 


In [13]:
from transformers import BlipProcessor, BlipForConditionalGeneration

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")


In [27]:
inputs = processor(img1, return_tensors="pt")
out = model.generate(**inputs, max_length=80)
caption = processor.decode(out[0], skip_special_tokens=True)

In [1]:
type(img1)

NameError: name 'img1' is not defined

In [28]:
print(caption)

a woman ' s long sleeved top with a scoop neck


In [26]:
text1

'solid dark blue fitted top in soft stretch jersey with a wide neckline and long sleeves'